###General Instructions
In this assignment, you will need to complete the code samples where indicated to accomplish the given objectives. **Be sure to run all cells** and export this notebook as an HTML with results included.  Upload the exported HTML file to Canvas by the assignment deadline.

####Assignment
Unlike previous exercises, you will not be provided any sample code from which to work.  You are given a starting point and an expected outcome.  Use what you have learned in class and in labs to complete the exercise as outlined below. Break your logic up into cells that tackle webpage retrieval, HTML parsing, and DataFrame assembly and persistence.

Board Game Geek is an online site that provides rankings for various boardgames.  A complete listing of ranked games is provided here: https://boardgamegeek.com/browse/boardgame

The data on this page is organized in a simple table format. The Title field is divided into the game's title and the year it was published. The title itself provides a link to more information on the game.

Write a routine to scrape this data from the page and persist it as a partitioned Parquet table named games.board_games.  Use the current date as the partitioning field.

The fields required in the final table are:

* Rank - the integer value from the Board Game Rank column
* ImageURL - the URL (as provided) of the image presented in the second column 
* Title - the title of the game
* YearPublished - the year the game was published
* GeekRating - the float value presented in the Geek Rating field
* AvgRating - the float value presented in the Avg Rating field
* NumVoters - the integer value presented in the NumVoters field

(You can ignore the Shop field)

While the Board Game Geek site has multiple pages of rankings, limit your scraping to the games presented on the page referenced above.  (In other words, don't hit the other 1000+ pages of rankings.)

Once you've populated your table, run the two SQL statements presented at the bottom of this notebook.  Save your notebook (with the results) as an HTML file and load it to Canvas.

In [4]:
# run this code to make sure beautiful soup is installed
dbutils.library.installPyPI('beautifulsoup4')
dbutils.library.restartPython()

In [5]:
# retrieve the web page
from urllib import request as url 
from bs4 import BeautifulSoup


request = url.Request('https://boardgamegeek.com/browse/boardgame')
try:
    response = url.urlopen(request)
except urllib.error.HTTPError as e:
    print('The HTTP status code returned is {0}'.format(e.getcode()) )
    print('The message returned is \'{0}\'\n'.format(e.msg) )
    print('The headers are the response are:\n{0}\n'.format(e.headers) )
    print('The content actually returned is:\n{0}'.format(e.fp.read().strip()))
    
html = response.read()

soup = BeautifulSoup(html)

In [6]:
# extract the data from the web page  
import re
from datetime import datetime

div = soup.find('div', attrs={'class':'table-responsive'})
tables = div.find_all('table')

table = soup.find('table', attrs={'class':'collection_table'})
table_rows = table.find_all('tr',attrs={'id':'row_'})
table_data = []
for table_row in table_rows:  
    Rank = table_row.find('td', attrs={'class':'collection_rank'}).find('a')['name']
    ImageURL = table_row.find('td', attrs={'class':'collection_thumbnail'}).find('img')['src']
    Title = table_row.find('td', attrs={'class':'collection_objectname'}).find('a').text
    YearPublished = table_row.find('td', attrs={'class':'collection_objectname'}).find('span').text[1:5]   
    [GeekRating,AvgRating,NumVoters]=[td.text.strip() for td in table_rows[1].find_all('td', attrs={'class':'collection_bggrating'})]
    row_data = [int(Rank), ImageURL, Title, int(YearPublished), float(GeekRating), float(AvgRating), float(NumVoters)]    
    table_data.append(row_data)


In [7]:
# convert the data to a Spark SQL table
from pyspark.sql.types import *
schema = StructType([
    StructField('Rank', IntegerType()),
    StructField('ImageURL', StringType()),
    StructField('Title', StringType()),
    StructField('YearPublished', IntegerType()),
    StructField('GeekRating', DoubleType()),
    StructField('AvgRating', DoubleType()),
    StructField('NumVoters', DoubleType())    
    ])

df = spark.createDataFrame(table_data, schema=schema)
spark.sql('create database if not exists games')
df_repart = df.repartition( 2, df.YearPublished)

df_repart.write.saveAsTable('games.board_games', partitionBy='YearPublished', mode='overwrite', format='parquet')

Execute the following SQL statements to validate your results:

In [9]:
%sql -- verify row count

SELECT COUNT(*) FROM games.board_games;

count(1)
100


In [10]:
%sql -- verify values

SELECT * FROM games.board_games ORDER BY rank ASC;

Rank,ImageURL,Title,GeekRating,AvgRating,NumVoters,YearPublished
1,https://cf.geekdo-images.com/micro/img/8JYMPXdcBg_UHddwzq64H4NBduY=/fit-in/64x64/pic2437871.jpg,Gloomhaven,8.467,8.62,36056.0,2017
2,https://cf.geekdo-images.com/micro/img/KbAKyhbG4qab4r-A_pBjUGvgal0=/fit-in/64x64/pic2452831.png,Pandemic Legacy: Season 1,8.467,8.62,36056.0,2015
3,https://cf.geekdo-images.com/micro/img/71v8jF5vZFBqE64QJ7S3UYtDAng=/fit-in/64x64/pic3536616.jpg,Terraforming Mars,8.467,8.62,36056.0,2016
4,https://cf.geekdo-images.com/micro/img/9f34JRDD1AhxnD62n5aX0rIW_g0=/fit-in/64x64/pic3490053.jpg,Brass: Birmingham,8.467,8.62,36056.0,2018
5,https://cf.geekdo-images.com/micro/img/APvZ_BYOt4ElpIXVl7i6wUp2BvM=/fit-in/64x64/pic2663291.jpg,Through the Ages: A New Story of Civilization,8.467,8.62,36056.0,2015
6,https://cf.geekdo-images.com/micro/img/D_wkWC_xeBStqd2Y4xXuUrDYpso=/fit-in/64x64/pic3727516.jpg,Twilight Imperium (Fourth Edition),8.467,8.62,36056.0,2017
7,https://cf.geekdo-images.com/micro/img/AgmwMDLQBFrLlV7aLvkjfWjzLIY=/fit-in/64x64/pic4325841.jpg,Star Wars: Rebellion,8.467,8.62,36056.0,2016
8,https://cf.geekdo-images.com/micro/img/uStZr7_8Tbthswp3sya1c2o1cs0=/fit-in/64x64/pic361592.jpg,Twilight Struggle,8.467,8.62,36056.0,2005
9,https://cf.geekdo-images.com/micro/img/Ifks_R9_4ObVmyHgN2j9lks-_8M=/fit-in/64x64/pic3763556.jpg,Gaia Project,8.467,8.62,36056.0,2017
10,https://cf.geekdo-images.com/micro/img/ZyNfyt5ymrWUmsudjtWnQ_H_hTI=/fit-in/64x64/pic4887376.jpg,Great Western Trail,8.467,8.62,36056.0,2016


In [11]:
%sql -- verify partitioning
DESCRIBE EXTENDED games.board_games;

col_name,data_type,comment
Rank,int,null
ImageURL,string,null
Title,string,null
GeekRating,double,null
AvgRating,double,null
NumVoters,double,null
YearPublished,int,null
# Partition Information,,
# col_name,data_type,comment
YearPublished,int,null
